In [10]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [34]:
class process:
    def __init__(self, size, timeUnit):
        self.size = size
        self.timeUnit = timeUnit
        self.remainTime = self.timeUnit

In [21]:
def clear():
    clear_output(wait=False)

In [46]:
def createTimeUnitTable(processes: list):
    processes = np.asarray(processes)
    df = pd.DataFrame(processes.reshape(-1,10))
    print(df)
    

In [48]:
processes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
createTimeUnitTable(processes)

    0   1   2   3   4   5   6   7   8   9
0   1   2   3   4   5   6   7   8   9  10
1  11  12  13  14  15  16  17  18  19  20
